In [1]:
import pandas as pd
import numpy as np
import csv
import os
%pprint off

Pretty printing has been turned OFF


In [2]:
path = '/Users/songheekim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
backups = next(os.walk(path))[1]
backups = [b for b in backups if b.startswith('backup')]
backups

['backup0_1_550', 'backup1_550-1050', 'backup3_0628', 'backup2_0610']

In [3]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [4]:
def return_res_catch_subj(backupfolder):
    
    path = '/Users/songheekim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
#     stimr = pd.read_csv(path+backupfolder+'/'+'wro1_stimuli.txt', delimiter="\t")
#     stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
#     stim['stim'] = stim['stim'].str[:-1]

    catchr = pd.read_csv(path+backupfolder+'/'+'wro1_catchresults.txt',delimiter="\t")
    catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})

    resr = pd.read_csv(path+backupfolder+'/'+'wro1_results.txt', delimiter="\t")
    res = resr.drop(columns=['stimtype_id','sense'])

    subj_raw = pd.read_csv(path+backupfolder+'/'+'wro1_subjects.txt',delimiter="\t")
    subj = subj_raw.rename(columns={"id": "subject_id"})

    return (res, catch, subj)

In [5]:
#### basic check on each dataset ##### 
b1 = backups[0]
b2 = backups[2]

for b in [b1, b2]:
    print('backup selected:', b)
    print(os.listdir(path+'/'+b))

    stimr = pd.read_csv(path+b+'/'+'wro1_stimuli.txt', delimiter="\t")
    stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
    stim['stim'] = stim['stim'].str[:-1]
    
    ques = pd.read_csv(path+b+'/'+'wro1_questions.txt', delimiter="\t")

    catchr = pd.read_csv(path+b+'/'+'wro1_catchresults.txt',delimiter="\t")
    catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})
    #print ("catch trial:", catch.shape[0])
    
    resr = pd.read_csv(path+b+'/'+'wro1_results.txt', delimiter="\t")
    res = resr.drop(columns=['stimtype_id','sense'])
    #print ("main trial:", res.shape[0])
    starttime = res.loc[res['id']==1, "start"].values[0]
    endtime = res.loc[res['id']==res.shape[0], "end"].values[0]
    print('FROM', starttime, 'TO', endtime)
    
    subj_raw = pd.read_csv(path+b+'/'+'wro1_subjects.txt',delimiter="\t")
    subj = subj_raw.rename(columns={"id": "subject_id"})
    #print('subject N:', subj.shape[0])
    
    print('# of questions:', ques.shape[0])
    print('# of catch HITs:', catch.shape[0])
    print('# of HITs:     ', res.shape[0])
    print('# of subjects: ', subj.shape[0])
    
    check1 = catch.shape[0] == res.shape[0]
    check2 = res.shape[0] >= subj.shape[0] 
    check3 = res['subject_id'].equals(other=catch['catch_subject_id'])

    if check1==0:
        print ('# of catch trials != # of trials')
    elif check2==0:
        print ('# of subjects are more than # of HITs')
    elif check3==0:
        print('subject_id are not the same in trials and catchtrials')
       
    print ('=============================================') 


backup selected: backup0_1_550
['.DS_Store', 'wro1_stimuli.txt', 'wro1_subjects.txt', 'wro1_catchresults.txt', 'wro1_stimtypes.txt', 'wro1_results.txt', 'wro1_questions.txt']
FROM 2021-05-25 16:26:41 TO 2021-05-28 01:18:52
# of questions: 72
# of catch HITs: 554
# of HITs:      554
# of subjects:  527
backup selected: backup3_0628
['wro1_stimuli copy.txt', '.DS_Store', 'wro1_stimuli.txt', 'wro1_subjects.txt', 'wro1_catchresults.txt', 'wro1_results.txt', 'wro1_questions.txt']
FROM 2021-05-28 16:52:09 TO 2021-06-23 20:57:21
# of questions: 72
# of catch HITs: 2262
# of HITs:      2262
# of subjects:  898


In [6]:
######catch trial info
catch1 = 'being an animal living on the surface of sun'
catch2 = 'being something about which you are supposed to answer questions in this assignment'
catch3 = 'being an object that is smaller than a shoe box and larger than a mountain'
catch4 = 'being something you have thought about within the last couple minutes'
catch5 = 'being used to check whether you are carefully reading the questions'
catch_answers = ['0/7','6','0','6','6']

def count_correct_answers(list1):
    '''list1 = correct answers, list2=actual responses'''
    catch_answers = ['0/7','6','0','6','6']
    if len(list1) != len(catch_answers):
        print ('responses have different lengths!')
        return
    correct_answer = 0
    if catch_answers[0] in list1[0].split('/'):
        correct_answer += 1
    for i in range(1,len(list1)):
        iscorrect = list1[i]==catch_answers[i]
        correct_answer += iscorrect        
    return correct_answer

count_correct_answers(['5', '6', '0', '6', '6'])

4

In [7]:
def merge_results(res, catch, subj): 
    res_all = pd.merge(left=res, right=subj, on='subject_id')
    res_all = res_all.sort_values(by=['id'])
    print('result df:', res.shape)
    print('subject df:', subj.shape)
    print('df returned:', res_all.shape)
    
    catch = catch.drop(columns='id') 
    res_all = pd.merge(left=res_all, right=catch, left_on="id", right_on="results_id")
    
    #print(res_all.shape)
    #print(catch.shape)
    print ('=========================================================')
    
    return res_all

In [8]:
## merge each data 
[res1, catch1, subj1] = return_res_catch_subj(b1)
[res2, catch2, subj2] = return_res_catch_subj(b2)
res_all1 = merge_results(res1, catch1, subj1)
res_all2 = merge_results(res2, catch2, subj2)

result df: (554, 9)
subject df: (527, 6)
df returned: (554, 14)
result df: (2262, 9)
subject df: (898, 6)
df returned: (2262, 14)


In [11]:
res_combined = pd.concat([res_all1, res_all2], ignore_index=True)
res_combined['id'] = list(range(1, res_combined.shape[0]+1))

print(res_all1.shape)
print(res_all2.shape)
print(res_combined.shape)
#res_combined

###check whether there is a duplicate row
# duplicates = res_combined.duplicated()
# duplicates = duplicates.index[duplicates]
# len(duplicates)


#### add lemma column
stimr = pd.read_csv(path+b1+'/'+'wro1_stimuli.txt', delimiter="\t")
stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
stim['stim'] = stim['stim'].str[:-1]
stim.rename(columns={'id':'stim_id'}, inplace=True)
#stim

res_combined = pd.merge(left=res_combined, right=stim, on="stim_id").sort_values(by="id")
res_combined = rearrange_col(res_combined, 'stim', 'stim_id')
res_combined

(554, 17)
(2262, 17)
(2816, 17)


,id,stim_id,stim,subject_id,response,start,end,rt,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response
0,1,159,laugh,1,1_0_0_3_0_0_1_5_6_2_3_3_6_2_0_0_0_0_0_0_0_0_2_...,2021-05-25 16:26:41,2021-05-25 16:35:15,10451_2940_2319_3999_2779_3110_6525_7596_3985_...,1_1_1_1_1_1_1_1_1_1_2_3_1_1_1_1_1_1_1_1_1_1_1_...,8545510,A3I9XLIHPPWPN1,f,57,18.0,Dataanalystadministrationassistant,1,1,0_6_0_6_6
10,2,295,see,2,6_3_0_1_1_2_0_4_0_1_0_3_3_0_0_0_0_0_0_0_0_0_0_...,2021-05-25 16:29:38,2021-05-25 16:36:24,4024_9148_2726_5130_5199_3487_2790_4329_4503_2...,1_4_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,2727964,A2CWJRAEFZ44HU,m,36,14.0,nursingassistantihelpoutpatientsandstaffinahos...,2,2,0_6_0_6_6
18,3,214,flatten,3,3_7_7_7_5_3_1_1_3_2_0_5_7_1_4_0_0_0_0_0_1_0_7_...,2021-05-25 16:26:58,2021-05-25 16:36:47,4432_2912_2592_3640_7976_6904_5368_6088_13600_...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_1_...,3205581,A7ERZELTAMWL5,m,70,18.0,PatrolofficerIridearoundaresidentialcomplextoc...,3,3,0_6_0_6_6
26,4,195,increase,4,4_3_6_4_3_2_3_3_6_5_5_5_6_6_5_5_6_3_6_6_4_6_6_...,2021-05-25 16:30:25,2021-05-25 16:37:35,2941_3771_4392_2726_3770_3690_2473_1306_1137_1...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_1_1_...,6434648,A1MKYMYY34DZIO,m,30,16.0,student,4,4,4_6_5_6_6
35,5,262,calm,5,4_0_0_0_3_0_0_1_4_0_5_0_4_2_2_2_0_1_0_0_0_0_3_...,2021-05-25 16:28:07,2021-05-25 16:38:09,4142_2349_7257_8933_4802_3372_3349_8304_4593_1...,1_1_1_1_1_1_1_1_1_1_2_1_2_1_2_2_1_1_1_1_1_1_1_...,1247441,A2LF84L3K71GR2,f,38,12.0,RestaurantSupervisor,5,5,0_6_0_6_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2812,241,specify,67,5_6_3_0_6_0_3_1_0_3_2_5_1_3_5_3_1_5_4_4_2_6_6_...,2021-06-23 16:53:51,2021-06-23 17:02:21,3708_4464_2788_2388_2708_4596_8992_2489_6216_2...,1_1_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_1_2_1_1_1_1_...,2268511,A2WPHVMLLEV5ZB,m,26,12.0,Mturker,2258,67,0_6_0_6_6
207,2813,80,wilt,280,0_0_0_5_4_0_0_3_3_0_3_3_0_0_0_0_0_0_0_0_0_1_0_...,2021-06-23 16:49:42,2021-06-23 17:02:27,15064_2207_9752_2816_6272_5664_4656_78280_1321...,1_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,9427636,A1P57CKX00HC2I,f,63,26.0,selfemployed,2259,280,0_6_0_6_6
555,2814,32,squeal,552,0_0_0_0_0_0_3_3_1_0_1_0_1_0_0_3_0_0_0_0_0_1_6_...,2021-06-23 17:01:15,2021-06-23 17:06:45,3222_1628_2109_2202_2187_2383_5384_2179_5352_4...,1_1_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_1_1_1_1_2_1_...,8197779,A2OVX9UW5WANQE,f,47,14.0,secretary,2260,552,0_6_0_6_6
1841,2815,221,darken,898,5_0_6_6_6_0_0_0_3_0_3_0_2_5_0_0_1_0_0_0_2_3_0_...,2021-06-23 17:06:28,2021-06-23 17:17:33,5103_16967_6294_4914_6429_4531_14854_6642_6963...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,3528996,A12MV66U9VGLQ1,f,81,19.0,retiredcomputerprogrammer,2261,898,7_6_7_6_6


In [ ]:
########test
# df = pd.DataFrame({'id':[1,2,3,4,5,6,7,8], 'stim_id': [1,2,3,1,2,3,7,6]})
# st = pd.DataFrame({'stim_id': [1,2,3,4,5,6,7,8,9], 'stim': ['a','b','c','d','e','f','g','h','i']})
# df

# st

# dfst = pd.merge(left=df, right=st, on="stim_id").sort_values(by="id")
# dfst

In [12]:
##### mark bad subjects
res_combined['catch_response_split'] = res_combined['catch_response'].apply(lambda x:x.split('_'))
catch_series = res_combined['catch_response_split'].tolist()
correct_answer = [count_correct_answers(l) for l in catch_series]
res_combined['correctN'] = correct_answer

res_combined.loc[res_combined['correctN']<4, 'badsubject'] = 'Y'
res_combined.loc[res_combined['correctN']>=4, 'badsubject'] = 'N'
res_bad = res_combined.loc[res_combined['badsubject']=='Y']
print('# of bad subject:', res_bad.shape[0], 'out of', res_combined.shape[0])

# of bad subject: 358 out of 2816


In [13]:
res_combined.drop(columns=['catch_response_split'], inplace=True)

In [14]:
## export to csv
res_combined.to_csv(path+'results_clean_0628.csv', index=False)

In [ ]:
#######to export bad subjects list
# df_out = res_combined.loc[res_combined['badsubject']=='Y', ]
# df_out.to_csv(path+'badsubjects.csv')